In [1]:
import os
import requests
from dotenv import load_dotenv
import base64
import json
import numpy as np
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
from utils.sf_conn import Config
from utils.assistant_api import Spotify_Assistant
from utils.prompt import PROMPT
import time
load_dotenv()

True

In [2]:
sf_config = Config()
create_conn = sf_config.create_sf_conn()

In [ ]:
cursor = create_conn.cursor()
query = "SELECT * FROM MAIN_SPOTIFY"
spotify_df = cursor.execute(query).fetch_pandas_all()
spotify_df.head(3)

In [ ]:
spotify_csv = spotify_df.to_csv('assistant_data/spotify_main_file')

In [3]:
def wait_until_run_status_completed(assistant, run_id):
    start_time = time.time() # starting a timer in case, the while loop gets stuck
    while True:
        (status := assistant.run_status(run_id))
        if status == "completed":
            break
        elif status in {"in_progress", "queued"}:
            cur_time = time.time()
            time_elasped = cur_time - start_time
            if time_elasped >= 600: # break the loop after it's been running for 10 mins
                break
            continue
        else:
            break
    assert status == "completed"

In [ ]:
# def upload_csv_file(file_path):
#     api_key =  os.getenv('OPENAI_API_KEY')
#     openai.api_key = api_key
#     with open(file_path, "rb") as file:
#         uploaded_file = openai.files.create(
#             file=file,
#             purpose='assistants'
#         )
#     return uploaded_file.id

In [4]:
assistant = Spotify_Assistant(assistant_id='asst_afQAdlE40WllyVRI7mDZBjha')
assistant.upload_csv_file('assistant_data/spotify_main_file')

# print(assistant.create_assistant(file_ids=[file_id]))
# assistant.attach_message_to_thread('What is the least popular song in the file?')
# run = assistant.start_run(override_prompt=PROMPT)
# assistant_status = wait_until_run_status_completed(assistant=assistant, run_id = run.id)
# response = assistant.assistant_answer()
# print(response)





'file-seTpgq57ZuMYjq83JfaADYjh'

In [ ]:
! pip list